In [ ]:
# 环境依赖安装和配置
%pip install anthropic openai python-dotenv requests sqlite3 pandas --quiet

import anthropic
import openai
import os
import sqlite3
import json
import time
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import httpx

# 加载环境变量
load_dotenv()

# 配置 Claude API 客户端（使用代理）
client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    http_client=httpx.Client(proxy="http://127.0.0.1:7890/")
)

print("✅ 环境配置完成!")
print("📋 已安装依赖: anthropic, openai, python-dotenv, requests, sqlite3, pandas")
print("🔑 API 配置: Claude API (流式输出模式)")
print("💾 记忆存储: SQLite 本地数据库")


In [ ]:
# SQLite 记忆存储系统 - 教学演示

def setup_memory_db():
    """
    初始化 SQLite 记忆数据库
    包含：对话历史、市场数据、用户偏好等表
    """
    conn = sqlite3.connect('agent_memory.db')
    cursor = conn.cursor()
    
    # 创建对话历史表
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS conversations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            role TEXT NOT NULL,
            content TEXT NOT NULL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
            session_id TEXT,
            task_type TEXT
        )
    ''')
    
    # 创建市场数据表（示例）
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS market_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            symbol TEXT NOT NULL,
            price REAL,
            change_percent REAL,
            volume INTEGER,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # 创建新闻摘要表
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS news_summaries (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            summary TEXT,
            sentiment TEXT,
            keywords TEXT,
            source TEXT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()
    return conn

def add_memory(conn, role, content, session_id="default", task_type="general"):
    """向记忆系统添加对话记录"""
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO conversations (role, content, session_id, task_type)
        VALUES (?, ?, ?, ?)
    ''', (role, content, session_id, task_type))
    conn.commit()
    
def get_recent_memory(conn, session_id="default", limit=5):
    """获取最近的对话记录"""
    cursor = conn.cursor()
    cursor.execute('''
        SELECT role, content, timestamp FROM conversations
        WHERE session_id = ?
        ORDER BY timestamp DESC LIMIT ?
    ''', (session_id, limit))
    return cursor.fetchall()

# 初始化记忆系统
memory_conn = setup_memory_db()
print("🗄️ SQLite 记忆数据库已初始化")

# 查看数据库表结构
cursor = memory_conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(f"📊 数据库表: {[table[0] for table in tables]}")

# 添加示例记忆
add_memory(memory_conn, "user", "分析平安银行最新股价走势", "demo_session", "trading_analysis")
add_memory(memory_conn, "assistant", "已获取平安银行(000001.SZ)最新数据，正在分析...", "demo_session", "trading_analysis")

# 展示记忆内容
recent_memories = get_recent_memory(memory_conn, "demo_session")
print("\n💭 最近记忆:")
for role, content, timestamp in recent_memories:
    print(f"  {role}: {content[:50]}... ({timestamp})")


In [ ]:
# Claude API 调用辅助函数（流式输出必需）

def call_claude_stream(prompt, system="", max_tokens=1000, temperature=0.7):
    """
    调用 Claude API 的辅助函数（流式输出）
    
    参数:
    - prompt: 用户输入的提示词
    - system: 系统提示词
    - max_tokens: 最大token数
    - temperature: 随机性参数（0-1，越高越随机）
    """
    full_response = ""
    
    try:
        with client.messages.stream(
            model="claude-3-5-sonnet-20241022",
            max_tokens=max_tokens,
            temperature=temperature,
            system=system,
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        ) as stream:
            for text in stream.text_stream:
                print(text, end="", flush=True)
                full_response += text
        
        print()  # 添加换行
        return full_response
        
    except Exception as e:
        print(f"❌ API 调用错误: {e}")
        return None

# 测试 Claude API 连接
print("🧪 测试 Claude API 连接...")
test_response = call_claude_stream(
    "请用一句话解释什么是AI Agent。",
    system="你是一个专业的AI技术讲解员，用简洁准确的语言回答问题。"
)

if test_response:
    print("✅ Claude API 连接成功！")
else:
    print("❌ Claude API 连接失败，请检查 API Key 和网络设置")


In [ ]:
# 模拟财经新闻数据
sample_financial_news = """
标题：央行降准释放流动性 银行股集体上涨
时间：2024-01-15 09:30
来源：财经日报

内容：中国人民银行昨日宣布下调存款准备金率0.5个百分点，释放长期资金约1万亿元。
此举旨在维护银行体系流动性合理充裕，支持实体经济发展。受此消息影响，今日开盘
银行板块集体上涨，其中平安银行涨幅达3.2%，招商银行上涨2.8%，工商银行涨2.1%。

市场分析师认为，降准政策将降低银行资金成本，有利于银行放贷投放，预计银行净息差
压力将有所缓解。同时，充裕的流动性也将推动债券市场走强，进一步支撑银行业绩表现。

风险提示：货币政策存在不确定性，银行股投资需关注利率变化和经济基本面。
"""

def news_extraction_workflow():
    """
    Step 1: 新闻信息提取
    从原始新闻中提取关键信息
    """
    print("🔍 步骤 1: 新闻信息提取")
    print("-" * 50)
    
    extraction_prompt = f"""
    请从以下财经新闻中提取关键信息，以JSON格式输出：

    新闻内容：
    {sample_financial_news}

    请提取：
    1. 事件类型（政策/业绩/市场动态等）
    2. 关键数字（涨跌幅、金额等）
    3. 影响板块/个股
    4. 市场情绪（正面/负面/中性）
    5. 主要观点
    """
    
    system_prompt = """
    你是专业的金融信息分析师。请准确提取新闻中的关键信息，
    格式要求：
    - 数字要准确
    - 股票代码要标准化（如：000001.SZ）
    - 情绪判断要客观
    """
    
    extracted_info = call_claude_stream(extraction_prompt, system_prompt, max_tokens=800)
    
    # 存储到记忆系统
    add_memory(memory_conn, "system", "新闻信息提取完成", "workflow_demo", "news_extraction")
    
    return extracted_info

# 执行新闻提取
extracted_data = news_extraction_workflow()


In [ ]:
def news_summarization_workflow(extracted_info):
    """
    Step 2: 智能摘要生成
    基于提取的信息生成专业的投资摘要
    """
    print("\n📝 步骤 2: 智能摘要生成")
    print("-" * 50)
    
    summarization_prompt = f"""
    基于以下提取的新闻信息，生成一份专业的银行内部投资简报：

    提取信息：
    {extracted_info}

    请生成包含以下部分的简报：
    1. 核心事件概述（2-3句话）
    2. 市场影响分析
    3. 相关标的表现
    4. 投资机会与风险
    5. 建议关注事项

    要求：
    - 语言专业但易懂
    - 突出数据和事实
    - 保持客观中性的态度
    """
    
    system_prompt = """
    你是银行投资研究部门的高级分析师，为内部客户经理提供专业的市场简报。
    写作风格：
    - 结构清晰，逻辑严密
    - 数据驱动，事实为准
    - 风险意识，合规优先
    """
    
    summary_report = call_claude_stream(summarization_prompt, system_prompt, max_tokens=1200)
    
    # 存储摘要到数据库
    if summary_report:
        cursor = memory_conn.cursor()
        cursor.execute('''
            INSERT INTO news_summaries (title, summary, sentiment, keywords, source)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            "央行降准释放流动性", 
            summary_report, 
            "正面", 
            "央行,降准,银行股,流动性", 
            "财经日报"
        ))
        memory_conn.commit()
    
    # 存储到记忆系统
    add_memory(memory_conn, "system", "投资简报生成完成", "workflow_demo", "news_summary")
    
    return summary_report

# 执行摘要生成
if extracted_data:
    summary_report = news_summarization_workflow(extracted_data)
else:
    print("⚠️ 新闻提取失败，跳过摘要生成步骤")


In [ ]:
def compliance_check_workflow(summary_report):
    """
    Step 3: 合规检查
    确保生成的内容符合银行监管要求
    """
    print("\n⚖️ 步骤 3: 合规检查")
    print("-" * 50)
    
    compliance_prompt = f"""
    请对以下投资简报进行合规检查，确保符合银行业监管要求：

    待检查内容：
    {summary_report}

    检查要点：
    1. 是否包含明确的风险提示？
    2. 是否避免了绝对化的投资建议？
    3. 是否强调了"仅供参考"的免责声明？
    4. 语言是否客观中性，避免煽动性表述？
    5. 是否遵循了"分析建议"而非"投资指令"的原则？

    请给出：
    - 合规性评分（1-10分）
    - 发现的问题
    - 修改建议
    - 最终合规版本
    """
    
    system_prompt = """
    你是银行合规部门的资深专员，熟悉金融监管法规和内控制度。
    检查标准：
    - 严格遵循《银行业监管指引》
    - 确保客户知情权和风险意识
    - 避免误导性投资建议
    - 保护银行和客户利益
    """
    
    compliance_result = call_claude_stream(compliance_prompt, system_prompt, max_tokens=1500)
    
    # 存储到记忆系统
    add_memory(memory_conn, "system", "合规检查完成", "workflow_demo", "compliance_check")
    
    return compliance_result

def finalize_report_with_disclaimer(content):
    """
    添加标准免责声明
    """
    disclaimer = """
    
    ⚠️ 重要免责声明：
    本分析报告基于公开市场信息，仅供内部参考，不构成投资建议。
    市场有风险，投资需谨慎。任何投资决策应基于客户自身风险承受能力和投资目标。
    银行不对因使用本报告信息而导致的投资损失承担责任。
    """
    
    return content + disclaimer

# 执行合规检查
if 'summary_report' in locals() and summary_report:
    compliance_result = compliance_check_workflow(summary_report)
    
    print("\n✅ Workflow 完整流程执行完毕")
    print("=" * 60)
    
    # 查看完整的记忆记录
    workflow_memories = get_recent_memory(memory_conn, "workflow_demo", 10)
    print("\n🧠 Workflow 执行记录:")
    for role, content, timestamp in workflow_memories:
        print(f"  [{timestamp}] {role}: {content}")
        
else:
    print("⚠️ 前序步骤失败，无法执行合规检查")


In [ ]:
# Agent 工具集定义

import random
from datetime import datetime, timedelta

def get_stock_data(symbol):
    """
    模拟获取股票实时数据
    
    参数：
    - symbol: 股票代码（如: 000001.SZ）
    
    返回：股票基本信息的JSON格式
    """
    # 模拟股票数据（实际应用中接入真实行情API）
    mock_data = {
        "000001.SZ": {
            "name": "平安银行",
            "price": round(15.38 + random.uniform(-0.5, 0.5), 2),
            "change": round(random.uniform(-0.3, 0.3), 2),
            "change_percent": round(random.uniform(-2, 2), 2),
            "volume": random.randint(10000000, 50000000),
            "market_cap": 297800000000,
            "pe_ratio": 4.85,
            "pb_ratio": 0.58,
            "industry": "银行"
        },
        "600036.SH": {
            "name": "招商银行", 
            "price": round(42.15 + random.uniform(-1, 1), 2),
            "change": round(random.uniform(-0.8, 0.8), 2),
            "change_percent": round(random.uniform(-2, 2), 2),
            "volume": random.randint(8000000, 30000000),
            "market_cap": 1200000000000,
            "pe_ratio": 5.2,
            "pb_ratio": 0.82,
            "industry": "银行"
        }
    }
    
    return mock_data.get(symbol, {"error": "股票代码未找到"})

def calculate_technical_indicators(symbol, period=20):
    """
    计算技术指标
    
    参数：
    - symbol: 股票代码
    - period: 计算周期（天数）
    
    返回：技术指标数据
    """
    # 模拟技术指标计算
    stock_data = get_stock_data(symbol)
    if "error" in stock_data:
        return stock_data
        
    # 模拟技术指标
    indicators = {
        "symbol": symbol,
        "period": period,
        "ma_5": round(stock_data["price"] * random.uniform(0.98, 1.02), 2),
        "ma_20": round(stock_data["price"] * random.uniform(0.95, 1.05), 2),
        "rsi": round(random.uniform(30, 70), 2),
        "macd": round(random.uniform(-0.5, 0.5), 3),
        "volume_ma": random.randint(15000000, 25000000),
        "support_level": round(stock_data["price"] * 0.95, 2),
        "resistance_level": round(stock_data["price"] * 1.05, 2)
    }
    
    return indicators

def get_market_sentiment():
    """
    获取市场整体情绪指标
    
    返回：市场情绪数据
    """
    sentiments = ["乐观", "中性", "谨慎", "悲观"]
    return {
        "overall_sentiment": random.choice(sentiments),
        "fear_greed_index": random.randint(20, 80),
        "volatility_index": round(random.uniform(15, 35), 2),
        "market_trend": random.choice(["上涨", "震荡", "下跌"]),
        "update_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

def search_financial_news(keyword, limit=3):
    """
    搜索相关财经新闻
    
    参数：
    - keyword: 搜索关键词
    - limit: 返回新闻数量
    
    返回：新闻列表
    """
    # 模拟新闻数据
    news_database = [
        {
            "title": "银行板块估值修复行情启动",
            "summary": "多家券商看好银行股投资价值",
            "sentiment": "正面",
            "publish_time": "2024-01-15 09:00",
            "relevance": 0.9
        },
        {
            "title": "央行货币政策保持稳健",
            "summary": "流动性合理充裕，支持实体经济",
            "sentiment": "中性",
            "publish_time": "2024-01-15 08:30",
            "relevance": 0.7
        },
        {
            "title": "外资增持银行股比例上升",
            "summary": "看好中国银行业长期发展前景",
            "sentiment": "正面", 
            "publish_time": "2024-01-14 16:20",
            "relevance": 0.8
        }
    ]
    
    # 简单匹配相关新闻
    relevant_news = [news for news in news_database if keyword.lower() in news["title"].lower()]
    return relevant_news[:limit] if relevant_news else news_database[:limit]

# 测试工具函数
print("🛠️ Agent 工具集测试")
print("-" * 40)

# 测试股票数据获取
stock_info = get_stock_data("000001.SZ")
print(f"📊 平安银行数据: {stock_info}")

# 测试技术指标
indicators = calculate_technical_indicators("000001.SZ")
print(f"📈 技术指标: RSI={indicators['rsi']}, MACD={indicators['macd']}")

# 测试市场情绪
sentiment = get_market_sentiment()
print(f"🎯 市场情绪: {sentiment['overall_sentiment']}, 恐慌贪婪指数: {sentiment['fear_greed_index']}")

print("\n✅ 所有工具函数测试完成！")


In [ ]:
# 交易趋势分析 Agent 实现

class TradingAnalysisAgent:
    """
    自主交易趋势分析智能体
    
    核心能力：
    1. 自主规划分析步骤
    2. 动态调用数据工具
    3. 综合多维度信息
    4. 生成投资建议
    """
    
    def __init__(self, memory_conn):
        self.memory_conn = memory_conn
        self.available_tools = {
            "get_stock_data": get_stock_data,
            "calculate_technical_indicators": calculate_technical_indicators,
            "get_market_sentiment": get_market_sentiment,
            "search_financial_news": search_financial_news
        }
        
    def analyze_stock(self, symbol, analysis_depth="standard"):
        """
        主分析函数 - Agent 自主执行分析流程
        
        参数:
        - symbol: 股票代码
        - analysis_depth: 分析深度（standard/deep）
        """
        print(f"🤖 启动交易趋势分析 Agent - 目标: {symbol}")
        print("=" * 60)
        
        # Step 1: Agent 规划分析策略
        planning_prompt = f"""
        作为专业的交易分析 Agent，我需要对股票 {symbol} 进行全面分析。
        
        请制定一个分析计划，包括：
        1. 需要获取哪些数据？
        2. 分析的优先级顺序？
        3. 重点关注的风险因素？
        4. 最终输出的建议类型？
        
        可用工具：
        - get_stock_data: 获取基本股价信息
        - calculate_technical_indicators: 计算技术指标
        - get_market_sentiment: 获取市场情绪
        - search_financial_news: 搜索相关新闻
        
        请制定一个详细的分析计划。
        """
        
        system_prompt = """
        你是一个专业的量化交易分析师，具备：
        1. 多年的A股市场经验
        2. 扎实的技术分析功底
        3. 敏锐的市场洞察力
        4. 严格的风险控制意识
        
        分析风格：数据驱动、逻辑严密、风险意识强
        """
        
        print("📋 Agent 正在制定分析计划...")
        analysis_plan = call_claude_stream(planning_prompt, system_prompt, max_tokens=800)
        
        # 存储分析计划到记忆
        add_memory(self.memory_conn, "agent", f"分析计划: {symbol}", "trading_agent", "planning")
        
        return self.execute_analysis_plan(symbol, analysis_plan)
    
    def execute_analysis_plan(self, symbol, plan):
        """
        执行分析计划
        """
        print(f"\n🔍 执行分析计划 - {symbol}")
        print("-" * 40)
        
        # 收集数据
        collected_data = {}
        
        # 1. 获取股票基础数据
        print("📊 获取股票基础数据...")
        collected_data["stock_data"] = get_stock_data(symbol)
        
        # 2. 计算技术指标
        print("📈 计算技术指标...")
        collected_data["technical_indicators"] = calculate_technical_indicators(symbol)
        
        # 3. 获取市场情绪
        print("🎯 获取市场情绪...")
        collected_data["market_sentiment"] = get_market_sentiment()
        
        # 4. 搜索相关新闻
        print("📰 搜索相关新闻...")
        stock_name = collected_data["stock_data"].get("name", symbol)
        collected_data["news"] = search_financial_news(stock_name)
        
        # 存储数据到 SQLite
        self.store_market_data(symbol, collected_data)
        
        # Agent 综合分析
        return self.comprehensive_analysis(symbol, collected_data)
    
    def store_market_data(self, symbol, data):
        """将收集的数据存储到 SQLite"""
        if "stock_data" in data and "error" not in data["stock_data"]:
            cursor = self.memory_conn.cursor()
            stock = data["stock_data"]
            cursor.execute('''
                INSERT INTO market_data (symbol, price, change_percent, volume)
                VALUES (?, ?, ?, ?)
            ''', (symbol, stock["price"], stock["change_percent"], stock["volume"]))
            self.memory_conn.commit()
    
    def comprehensive_analysis(self, symbol, collected_data):
        """
        Agent 进行综合分析
        """
        print(f"\n🧠 Agent 正在进行综合分析...")
        print("-" * 40)
        
        # 构建分析提示词
        analysis_prompt = f"""
        基于以下收集的数据，对股票 {symbol} 进行全面的交易趋势分析：

        股票基础数据：
        {json.dumps(collected_data["stock_data"], ensure_ascii=False, indent=2)}

        技术指标：
        {json.dumps(collected_data["technical_indicators"], ensure_ascii=False, indent=2)}

        市场情绪：
        {json.dumps(collected_data["market_sentiment"], ensure_ascii=False, indent=2)}

        相关新闻：
        {json.dumps(collected_data["news"], ensure_ascii=False, indent=2)}

        请提供一份专业的分析报告，包括：
        1. 当前股价走势评估
        2. 技术指标信号解读
        3. 市场情绪影响分析
        4. 基本面新闻解读
        5. 综合投资建议（买入/持有/卖出/观望）
        6. 风险提示
        7. 关键价位参考

        要求：
        - 结论明确但不绝对化
        - 充分考虑风险因素
        - 提供具体的操作建议
        - 包含止损止盈位参考
        """
        
        system_prompt = """
        你是银行投资顾问部门的首席分析师，为高净值客户提供专业投资建议。
        
        分析原则：
        1. 数据优先：所有结论基于具体数据
        2. 风险意识：充分考虑各种风险因素  
        3. 客观中性：避免过度乐观或悲观
        4. 实用性强：提供可执行的投资建议
        5. 合规要求：强调风险提示和免责声明
        """
        
        analysis_result = call_claude_stream(analysis_prompt, system_prompt, max_tokens=2000)
        
        # 存储分析结果
        add_memory(self.memory_conn, "agent", f"完成 {symbol} 综合分析", "trading_agent", "analysis_complete")
        
        # 添加免责声明
        final_result = self.add_compliance_disclaimer(analysis_result)
        
        return final_result
    
    def add_compliance_disclaimer(self, analysis):
        """添加合规免责声明"""
        disclaimer = """
        
        ⚠️ 重要声明：
        • 本分析基于公开市场数据，仅供投资参考，不构成投资建议
        • 股市有风险，投资需谨慎，过往业绩不代表未来表现
        • 任何投资决策应基于个人风险承受能力和财务状况
        • 建议咨询专业投资顾问，并进行充分的风险评估
        • 本行不对因使用此分析而产生的投资损失承担责任
        """
        return analysis + disclaimer

# 创建并测试 Trading Agent
trading_agent = TradingAnalysisAgent(memory_conn)

print("🚀 Trading Analysis Agent 已创建")
print("准备分析平安银行 (000001.SZ)...")


In [ ]:
# 执行 Trading Agent 分析

print("🎯 开始执行交易趋势分析...")
print("目标股票: 平安银行 (000001.SZ)")
print("=" * 60)

# 执行完整的 Agent 分析流程
analysis_result = trading_agent.analyze_stock("000001.SZ")

print("\n" + "="*60)
print("✅ Trading Agent 分析完成！")

# 查看 Agent 执行记录
print("\n🧠 Agent 执行记录:")
agent_memories = get_recent_memory(memory_conn, "trading_agent", 5)
for role, content, timestamp in agent_memories:
    print(f"  [{timestamp}] {role}: {content}")

# 查看存储的市场数据
print("\n📊 存储的市场数据:")
cursor = memory_conn.cursor()
cursor.execute("SELECT * FROM market_data ORDER BY timestamp DESC LIMIT 3")
market_records = cursor.fetchall()
for record in market_records:
    print(f"  ID:{record[0]} | {record[1]} | 价格:{record[2]} | 变化:{record[3]}% | 成交量:{record[4]}")
    
print(f"\n📈 分析报告已生成，共 {len(analysis_result) if analysis_result else 0} 字符")


In [ ]:
# 多智能体协作示例：银行板块综合分析

class MultiAgentOrchestrator:
    """
    多智能体协调器
    模拟 Anthropic 的 Orchestrator-Workers 模式
    """
    
    def __init__(self, memory_conn):
        self.memory_conn = memory_conn
        self.workers = {}
        
    def create_specialist_agent(self, name, specialty):
        """创建专业领域的子智能体"""
        self.workers[name] = {
            "specialty": specialty,
            "status": "ready",
            "results": None
        }
        
    def bank_sector_analysis(self, bank_symbols):
        """
        银行板块综合分析 - 多智能体协作示例
        """
        print("🎛️ 启动多智能体协调器")
        print("任务：银行板块综合分析")
        print(f"目标股票：{bank_symbols}")
        print("=" * 50)
        
        # 1. 协调器制定分工策略
        orchestration_prompt = f"""
        作为多智能体系统的协调器，我需要分析银行板块的多只股票：{bank_symbols}
        
        请设计一个高效的分工方案：
        1. 如何分配这些股票给不同的专家智能体？
        2. 每个专家智能体应该专注什么方面？
        3. 如何保证分析的全面性和一致性？
        4. 最终如何汇总各专家的分析结果？
        
        专家智能体类型：
        - 技术分析专家：专注技术指标和图表形态
        - 基本面分析专家：专注财务数据和行业动态  
        - 市场情绪专家：专注资金流向和市场情绪
        - 风险控制专家：专注风险评估和合规检查
        
        请制定详细的协作方案。
        """
        
        print("📋 协调器正在制定分工方案...")
        coordination_plan = call_claude_stream(
            orchestration_prompt, 
            "你是专业的投资研究团队负责人，擅长协调多个专家的工作。",
            max_tokens=1000
        )
        
        # 2. 并行执行专家分析
        expert_results = self.parallel_expert_analysis(bank_symbols)
        
        # 3. 汇总分析结果
        final_report = self.synthesize_results(bank_symbols, expert_results)
        
        return final_report
    
    def parallel_expert_analysis(self, symbols):
        """
        模拟多个专家智能体并行分析
        """
        print("\\n👥 启动专家智能体并行分析...")
        print("-" * 40)
        
        expert_results = {}
        
        for symbol in symbols:
            print(f"\\n📊 分析 {symbol}:")
            
            # 获取股票数据
            stock_data = get_stock_data(symbol)
            technical_data = calculate_technical_indicators(symbol)
            
            # 模拟不同专家的分析视角
            experts = {
                "技术分析专家": {
                    "focus": "技术指标和趋势",
                    "data": technical_data,
                    "perspective": "从技术面角度"
                },
                "基本面专家": {
                    "focus": "估值和财务健康度", 
                    "data": stock_data,
                    "perspective": "从基本面角度"
                },
                "市场情绪专家": {
                    "focus": "市场情绪和资金流向",
                    "data": get_market_sentiment(),
                    "perspective": "从市场情绪角度"
                }
            }
            
            symbol_results = {}
            
            for expert_name, expert_info in experts.items():
                print(f"  💭 {expert_name}分析中...")
                
                expert_prompt = f\"\"\"
                作为{expert_name}，请{expert_info["perspective"]}分析股票 {symbol}：
                
                专注领域：{expert_info["focus"]}
                
                相关数据：
                {json.dumps(expert_info["data"], ensure_ascii=False, indent=2)}
                
                请提供简洁的专业意见（100字以内）：
                1. 关键发现
                2. 投资建议 
                3. 风险提示
                \"\"\"
                
                expert_analysis = call_claude_stream(
                    expert_prompt,
                    f"你是银行业的{expert_name}，具有丰富的{expert_info['focus']}分析经验。",
                    max_tokens=300
                )
                
                symbol_results[expert_name] = expert_analysis
                
            expert_results[symbol] = symbol_results
            
        return expert_results
    
    def synthesize_results(self, symbols, expert_results):
        """
        协调器汇总所有专家意见
        """
        print("\\n🔮 协调器正在汇总专家意见...")
        print("-" * 40)
        
        synthesis_prompt = f\"\"\"
        作为投资研究团队的负责人，我收到了各位专家对银行板块的分析意见：
        
        分析目标：{symbols}
        
        专家意见汇总：
        {json.dumps(expert_results, ensure_ascii=False, indent=2)}
        
        请整合所有专家意见，形成一份银行板块的综合投资报告：
        
        1. 板块整体评估
        2. 个股对比分析
        3. 投资机会排序
        4. 风险因素总结
        5. 配置建议
        6. 关注要点
        
        要求：
        - 平衡各专家意见
        - 突出一致性观点
        - 标明分歧领域
        - 提供明确建议
        \"\"\"
        
        final_report = call_claude_stream(
            synthesis_prompt,
            "你是资深的投资研究总监，善于整合不同专家的观点形成全面的投资策略。",
            max_tokens=2000
        )
        
        # 存储到记忆系统
        add_memory(self.memory_conn, "orchestrator", "银行板块多智能体分析完成", "multi_agent", "synthesis")
        
        return final_report

# 创建多智能体系统
orchestrator = MultiAgentOrchestrator(memory_conn)

# 分析银行板块主要股票
bank_stocks = ["000001.SZ", "600036.SH"]  # 平安银行、招商银行

print("🚀 启动银行板块多智能体分析系统...")
multi_agent_result = orchestrator.bank_sector_analysis(bank_stocks)


In [ ]:
# 清理和总结

print("🧹 清理资源和数据...")

# 查看本次学习的完整记忆记录
print("\n📚 本次学习记录总结:")
print("=" * 50)

# 按任务类型分组显示记忆
task_types = ["news_extraction", "news_summary", "compliance_check", "trading_agent", "multi_agent"]

for task_type in task_types:
    cursor = memory_conn.cursor()
    cursor.execute('''
        SELECT role, content, timestamp FROM conversations
        WHERE task_type = ?
        ORDER BY timestamp
    ''', (task_type,))
    
    records = cursor.fetchall()
    if records:
        print(f"\n🔸 {task_type.upper()} 相关记录:")
        for role, content, timestamp in records:
            print(f"  [{timestamp}] {role}: {content}")

# 统计数据库内容
print("\n📊 数据库统计:")
cursor = memory_conn.cursor()

# 对话记录统计
cursor.execute("SELECT COUNT(*) FROM conversations")
conversation_count = cursor.fetchone()[0]
print(f"  对话记录: {conversation_count} 条")

# 市场数据统计
cursor.execute("SELECT COUNT(*) FROM market_data")
market_data_count = cursor.fetchone()[0]
print(f"  市场数据: {market_data_count} 条")

# 新闻摘要统计
cursor.execute("SELECT COUNT(*) FROM news_summaries")
news_count = cursor.fetchone()[0]
print(f"  新闻摘要: {news_count} 条")

print("\n✅ AI Agent 工作流基础教程完成!")
print("🎓 恭喜您已掌握银行业 AI Agent 开发的核心技能!")

print("\n" + "="*60)
print("📖 推荐进阶学习路径:")
print("1. 深入学习 Prompt 工程 -> prompt/02_Prompt工程基础教程.ipynb")
print("2. 掌握工具使用技巧 -> tool_use/01_工具使用.ipynb") 
print("3. 了解模型微调技术 -> fineture/03_模型微调基础教程.ipynb")
print("4. 探索结构化输出 -> prompt/04_结构化输出基础教程.ipynb")

# 关闭数据库连接
memory_conn.close()
print("\n🔐 数据库连接已安全关闭")
